# resume Critique

## Notebook Purpose
resume-Critique Evaluation
 
## Tasks
- Evaluation

## Notable TODOs:
--

# Setup

### Path

In [2]:
import sys
from pathlib import Path

NB_FOLDER = 'notebooks'
NB_PATH = Path.cwd()
B_PATH = NB_PATH.parent

main_spaths = ['scripts', 'data']
main_paths = [(B_PATH / m_spath) for m_spath in main_spaths]
data_spaths = ['external', 'processed', 'raw', 'results', 'training']
data_paths = [(B_PATH / 'data' / data_spath) for data_spath in data_spaths]
project_paths = main_paths + data_paths

if NB_PATH.stem == NB_FOLDER:
	# Create Project Folders and Set Paths
	for p_path in project_paths:
		p_path.mkdir(exist_ok=True, parents=True)
		p_name = p_path.name
		v_name = f"DATA_{p_name}_PATH" if p_path.parent.name == 'data' else f"{p_name}_PATH"
		globals()[v_name.upper()] = p_path

	# Custom Paths

	# Append SCRIPTS as Python Path
	sys.path.append(SCRIPTS_PATH.as_posix())

### Installation

In [ ]:
!git lfs install

In [ ]:
pip install json_repair

In [ ]:
pip install transformers

In [ ]:
pip install joblib

In [ ]:
pip install numpy

In [ ]:
pip install jsonschema

In [ ]:
pip install to-json-schema

In [ ]:
!pip install \
	--extra-index-url=https://pypi.nvidia.com \
	cudf-cu12==24.8.3 cuml-cu12==24.8.0 \
	cugraph-cu12==24.8.0

In [ ]:
pip install sentence-transformers

In [ ]:
pip install seaborn

In [ ]:
conda install tsnecuda

In [ ]:
pip install openai

In [ ]:
pip install whatlang-pyo3

In [ ]:
pip install tiktoken

### System Infomation

In [3]:
!bash {SCRIPTS_PATH.as_posix()}/notebook_info.sh

## GLOBAL INFO
Conda Python Version: 3.12.2.final.0
Conda Base Path: /opt/miniconda3
Conda Base Version: 24.9.2

## ENVIRONMENT INFO
Active Environment: cond_resume_critique_env
Environment Python Version: Python 3.10.15
Environment Python Path: /opt/miniconda3/envs/cond_resume_critique_env/bin/python
Environment IPython Version: 8.29.0
Environment IPykernel Version: 6.29.5

## GPU INFO:
CUDA Device Initialized Traceback (most recent call last):
  File "/opt/miniconda3/envs/cond_resume_critique_env/bin/numba", line 5, in <module>
    from numba.misc.numba_entry import main
  File "/opt/miniconda3/envs/cond_resume_critique_env/lib/python3.10/site-packages/numba/__init__.py", line 59, in <module>
    _ensure_critical_deps()
  File "/opt/miniconda3/envs/cond_resume_critique_env/lib/python3.10/site-packages/numba/__init__.py", line 45, in _ensure_critical_deps
    raise ImportError(msg)
ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.


GPU Info: Thu Dec 19 11:31:41 2024       
+

### Import

In [4]:
## Extensions
if 'autoreload' not in get_ipython().extension_manager.loaded:
	%load_ext autoreload
# %load_ext cudf.pandas
%load_ext dotenv
%autoreload 2

In [5]:
import pandas as pd
import warnings
import torch
from save_utils import save_obj, load_obj, run_api

### Configuration

In [6]:
# Pandas
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Future Warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

# Env+
%dotenv {(B_PATH / '.env').as_posix()}

# Torch Version
print(f"Torch Version: {torch. __version__}")

# Torch Cuda Available
torch.cuda.is_available()

Torch Version: 2.5.1+cu124


True

In [7]:
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()
@register_cell_magic
def pybash(line, cell):
	ipython.run_cell_magic('bash', '', cell.format(**globals()))

## Evaluation

In [8]:
results = {int(path.stem.split("_")[-1]): load_obj(path) for path in sorted((DATA_RESULTS_PATH / "run_2").glob("*.pkl"))}

In [9]:
for model_n, result in results.items():
	print(model_n)
	print(result.keys())

1
dict_keys(['0.45', '0.60', '0.75'])
2
dict_keys(['0.45', '0.60', '0.75'])
3
dict_keys(['0.45', '0.60', '0.75'])
4
dict_keys(['0.45', '0.60', '0.75'])
5
dict_keys(['0.45', '0.60', '0.75'])


## Joining Results and Ground Truth

In [10]:
results[1]

defaultdict(list,
            {'0.45': ['The content of your resume requires some refinement to fully capture the attention of potential employers. As it stands, while you do share valuable experiences, there is room for improvement in how these experiences are articulated. Presenting your professional history in a more engaging and impactful way will significantly enhance your chances of making a memorable impression.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n### Requested by: Dimeji Ajasin\t| dimejiajasin@gmail.com\t| (206)-606-8899\n\n### Date requested: October 17th, 2024\t  | Location: Seattle, WA\n\n### Thank You,\n\nResume Evaluation Team\n\n### Note: This is not a job offer or solicitation.',
              "The quality of content in your resume is the foundation for making a strong impression on potential employers. While you have a solid professional history, there's a clear need for improvement in how it’s articulated. A well-crafted resume should not only lis

In [13]:
results_recs = {}
for model_n, result in results.items():
	df = pd.read_json(DATA_TRAINING_PATH / f"resume_critique_model_{model_n}.jsonl", orient="records", lines=True)
	df_sample = df.sample(50, random_state=111)
	inputs = df_sample.input.tolist()
	outputs = df_sample.output.tolist()
	output_pred_1 = result["0.45"]
	output_pred_2 = result["0.60"]
	output_pred_3 = result["0.75"]

	recs = []
	for input, output_gt, output_pred_1, output_pred_2, output_pred_3 in zip(inputs, outputs, output_pred_1, output_pred_2, output_pred_3):
		recs.append({
			"input": input,
			"output_gt": output_gt,
			"output_pred_1": output_pred_1,
			"output_pred_2": output_pred_2,
			"output_pred_3": output_pred_3
		})
	results_recs[model_n] = recs

In [14]:
results_recs[4]

[{'input': 'Dimeji Ajasin   | Project Manager/ Architectural Designer\ndimejiajasin@gmail.com\t* +1 (425) 505 - 7582\nLink to portfolio\n\nOBJECTIVE\n\nProject Manager/ Architect with industry experience. Enjoy creative problem-solving and\ngetting exposure to multiple projects, and I would excel in the collaborative environment\nin which your company prides itself. Life-long learner looking for new exposure to new\nexperiences.\n\n\nWORK EXPERIENCE\n\nIHB Architects | Project Manager / Architectural Designer\t04/2020 - Present\nKent, Washington\n\n● Manage project/ program activities and multiple priorities in various project sizes.\nMeeting project deadlines and requirements.\n● Developed and presented design concepts to clients, resulting in a 90%\nsatisfaction rate and a 15% increase in repeat business.\n● Oversee the processing of submittals, requests for information, shop drawings,\nsubstitution requests, change proposals, and mods\n● Prepare responses to City comments.\n● Contri

In [15]:
results_recs[1]

[{'input': 'Dimeji Ajasin   | Project Manager/ Architectural Designer\ndimejiajasin@gmail.com\t* +1 (425) 505 - 7582\nLink to portfolio\n\nOBJECTIVE\n\nProject Manager/ Architect with industry experience. Enjoy creative problem-solving and\ngetting exposure to multiple projects, and I would excel in the collaborative environment\nin which your company prides itself. Life-long learner looking for new exposure to new\nexperiences.\n\n\nWORK EXPERIENCE\n\nIHB Architects | Project Manager / Architectural Designer\t04/2020 - Present\nKent, Washington\n\n● Manage project/ program activities and multiple priorities in various project sizes.\nMeeting project deadlines and requirements.\n● Developed and presented design concepts to clients, resulting in a 90%\nsatisfaction rate and a 15% increase in repeat business.\n● Oversee the processing of submittals, requests for information, shop drawings,\nsubstitution requests, change proposals, and mods\n● Prepare responses to City comments.\n● Contri

## Defining EOS

In [18]:
for rec in results_recs[4]:
	print("OUTPUT PRED 1:")
	print(rec["output_pred_1"])
	print()
	print("OUTPUT PRED 2:")
	print(rec["output_pred_2"])
	print()
	print("OUTPUT PRED 3:")
	print(rec["output_pred_3"])
	print()
	print("OUTPUT GT:")
	print(rec["output_gt"])
	print("-"*100)
	print("-"*100)
	print("-"*100)

OUTPUT PRED 1:
'Introduction': Your work history section includes substantial content that goes beyond mere job titles and company names. However, the way this information is currently presented could be enhanced to make your accomplishments more engaging and impactful. An effective work history should not only list your responsibilities but also highlight the results of your work, showcasing the value you brought to each role.
'Achiever': Your work history does a commendable job of detailing both tasks and accomplishments. However, it’s important to strike a balance between the two, ensuring that your achievements are not overshadowed by a focus on duties. Using accomplishment-driven language will help present you as an 'achiever' rather than just a 'doer,' showcasing your measurable impact in previous roles.
'Achievements': ["Example of task-based language: 'Assisted in the preparation of construction documents and specifications.'", "Example of achievement-driven language: 'Enhanced

In [ ]:
eos = {}
eos[1] = ["{'", "<|eot_id|>", "\n", " I ", " I'm ", "let's"]
eos[4] = ["['", "'", "']", " \\n", "###"]

In [49]:
for temp, outputs in results[3].items():
	print(temp)
	print()
	for output in outputs:
		print(output)
		print("-"*100)
		
	print("-"*100)
	print("-"*100)
	print("-"*100)

0.45

Highlighting key skills and competencies is vital for ensuring your resume resonates with potential employers. If your document does not clearly list relevant hard skills, it's essential to add a dedicated section that emphasizes these skills. This list should align with your career goals, making it easy for hiring managers to identify your qualifications at a glance.
----------------------------------------------------------------------------------------------------
Having a dedicated section for key skills and competencies is essential. This section should emphasize hard skills that directly align with your career goals. If you don’t already have such a list, consider adding one to strengthen your resume's appeal. Highlighting relevant skills will ensure that your resume resonates with potential employers. This section should be tailored to reflect the specific requirements of the roles you are targeting. By including a targeted list of skills, you can effectively communicate y